In [1]:
def get_validation_generator(eval_batch_size=1):
    validation_dir = "D:/GoogleDrive/Machine_Learning_Project-Classification+SRCNN/secondTaskData/validation" #change with your path (sharedFolder->secondTaskData->validationion
    transform_dir = "D:/GoogleDrive/Machine_Learning_Project-Classification+SRCNN/secondTaskData/transform" #change with your path (sharedFolder->secondTaskData->transform
    flip = True
    test_datagen = ImageDataGenerator(rescale=1. / 255)
    test_transform_datagen = ImageDataGenerator(rescale=1. / 255)
    random = True
    seed = 42
    shape = (256, 256)
    shape_hr = (1024, 1024)
    validation_generator_x = test_transform_datagen.flow_from_directory(
        transform_dir,
        target_size=shape,
        batch_size=eval_batch_size,
        class_mode=None, shuffle=True, seed=seed)
    validation_generator_y = test_datagen.flow_from_directory(
        validation_dir,
        target_size=shape_hr,
        batch_size=eval_batch_size,
        class_mode=None, shuffle=True, seed=seed)
    validation_generator = zip(validation_generator_x, validation_generator_y)
    return validation_generator


In [2]:
def get_train_generator(train_batch_size=1):
    train_dir = "D:/GoogleDrive/Machine_Learning_Project-Classification+SRCNN/secondTaskData/training" #change with your path (sharedFolder->secondTaskData->training
    transform_train_dir = "D:/GoogleDrive/Machine_Learning_Project-Classification+SRCNN/secondTaskData/transform_train" #change with your path (sharedFolder->secondTaskData->transform_train
    flip = True
    # encode the images in float vector. 
    train_datagen = ImageDataGenerator(rescale=1. / 255,
                                       horizontal_flip=flip, vertical_flip=flip)

    train_transform_datagen = ImageDataGenerator(rescale=1. / 255,
                                                 horizontal_flip=flip, vertical_flip=flip)

    random = True
    seed = 42
    shape = (256, 256)
    shape_hr = (1024, 1024)
    train_generator_x = train_transform_datagen.flow_from_directory(
        transform_train_dir,
        target_size=shape,
        batch_size=train_batch_size,
        class_mode=None, shuffle=True, seed=seed)
    train_generator_y = train_datagen.flow_from_directory(train_dir,
                                                          target_size=shape_hr,
                                                          batch_size=train_batch_size,
                                                          class_mode=None, shuffle=True, seed=seed)

    train_generator = zip(train_generator_x, train_generator_y)
    return train_generator


In [3]:
from keras.preprocessing.image import ImageDataGenerator

from skimage import img_as_ubyte

def get_image_generators(train_batch_size=1, eval_batch_size=1):
    train_generator = get_train_generator(train_batch_size)
    validation_generator = get_validation_generator(eval_batch_size)
    return train_generator, validation_generator


Using TensorFlow backend.


In [4]:
from keras import layers
from keras import models
from keras.optimizers import Adam

def CNN(create_model=True):
    if create_model:
        #make generator
        print("CREATION MODEL")
        generatorInput = layers.Input(shape=(256, 256, 3))
        generator = layers.Conv2D(64, (9, 9), input_shape=(256, 256, 3), padding='same', activation='relu')(
            generatorInput)  #n1

        generator = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(generator)
        generator = layers.BatchNormalization(momentum=0.8)(generator)

        generator = layers.Conv2D(64, (3, 3), padding='same')(generator)
        generator = layers.BatchNormalization(momentum=0.8)(generator)

        #loop
        for a in range(7):
            generator = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(generator)
            generator = layers.BatchNormalization(momentum=0.8)(generator)

        generator = layers.Conv2D(64, (3, 3), padding='same', activation='relu', name='exitLoop')(generator)
        generator = layers.BatchNormalization(momentum=0.8)(generator)

        generator = layers.Conv2DTranspose(128, 3, strides=2, padding='same')(generator)
        generator = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(generator)

        generator = layers.Conv2DTranspose(64, 3, strides=2, padding='same')(generator)
        generator = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(generator)

        generatorOutput = layers.Conv2D(3, (9, 9), padding='same', activation='tanh')(generator)

        gen_model = models.Model(generatorInput, generatorOutput, name="generator")

        print("generator")
        gen_model.summary()
        lossToUse = 'mse'
        optimizerToUse = Adam(0.0001, 0.5)
        gen_model.compile(optimizer=optimizerToUse, loss=lossToUse)
        return gen_model
    else:
        print("LOADING MODEL")
        #gen_model = models.load_model('D:/GoogleDrive/Machine_Learning_Project-Classification+SRCNN/secondTaskData/temporaryModel.h5')
        path = "D:/GoogleDrive/Machine_Learning_Project-Classification+SRCNN/secondTaskData/"  #change with your path (sharedFolder->secondTaskData)
        gen_model = models.load_model(path + 'GeneratorFitNew.h5')
    return gen_model


In [5]:
def save_model(model, path="", name="name"):
    model.save(path + name + '.h5')
    model.save_weights(path + name + 'Weights.h5')
    model_json = model.to_json()
    with open(path + name + '.json', "w") as json_file:
        json_file.write(model_json)
    from keras.utils.vis_utils import plot_model
    plot_model(model, to_file=path + name + 'Model.png', show_shapes=True, show_layer_names=True)

    from contextlib import redirect_stdout

    with open(path + name + 'Parameter.txt', "w") as model_summary:
        with redirect_stdout(model_summary):
            model.summary()


In [6]:
import os

import imageio
import matplotlib.pyplot as plt
from skimage import img_as_ubyte

def save_result(numberEpoch=0, path=""):
    os.makedirs(path + 'result/' + str(numberEpoch),
                exist_ok=True)
    path_pure = path + 'PureImage'
    os.makedirs(path_pure, exist_ok=True)
    t, v = get_image_generators()
    for i in range(100):
        x_to_transform, y_hight = next(v)
        generated = model.predict(x_to_transform)
        img_hr = img_as_ubyte(y_hight[0])
        img_to_transform = img_as_ubyte(x_to_transform[0])
        img_generated = img_as_ubyte(generated[0])
        f, axarr = plt.subplots(1, 3)
        plt.rcParams["figure.figsize"] = [20, 9]
        axarr[0].set_title("Original LR")
        axarr[0].grid(False)
        axarr[0].imshow(img_to_transform, aspect='auto')
        axarr[1].set_title("Generated HR")
        axarr[1].grid(False)
        axarr[1].imshow(img_generated, aspect='auto')
        axarr[2].set_title("HR")
        axarr[2].grid(False)
        axarr[2].imshow(img_hr, aspect='auto')
        f.savefig(path + 'result/' + str(numberEpoch) + '/' + str(
            numberEpoch) + '(' + str(i) + ').png', dpi=150, bbox_inches='tight')
        plt.close(f)
        imageio.imwrite(path_pure + '/' + str(numberEpoch) + '(' + str(i) + ').png', img_generated)
        print("saved image ============>", i)


In [7]:
from keras.callbacks import Callback


class PartialSaver(Callback):
    def __init__(self, save_step, previousStep, path):
        self.v = get_validation_generator()
        self.batch_seen = 100
        self.pred_step = save_step
        self.previous = previousStep
        self.path = path
        self.path_res = self.path + "PartialResults"
        self.path_pure = self.path + 'PartialResults/PureImage'
        os.makedirs(self.path_res, exist_ok=True)
        os.makedirs(self.path_pure, exist_ok=True)

def on_epoch_end(self, epoch, logs={}):
    path = self.path
    if epoch % 1000 == 0:
        if epoch > 999:
            print("1000 epochs completed")
        else:
            print("STARTED")
        print("EPOCH: ", epoch, " with loss values: ", logs)

    if epoch % self.pred_step == 0:
        x_to_transform, y_hight = next(self.v)
        self.batch_seen -= 1
        generated = model.predict(x_to_transform)
        img_to_transform = img_as_ubyte(x_to_transform[0])
        img_hr = img_as_ubyte(y_hight[0])
        img_generated = img_as_ubyte(generated[0])
        plt.rcParams["figure.figsize"] = [20, 9]
        f, axarr = plt.subplots(1, 3)
        axarr[0].set_title("Original LR")
        axarr[0].grid(False)
        axarr[0].imshow(img_to_transform, aspect='auto')
        axarr[1].set_title("Generated HR")
        axarr[1].grid(False)
        axarr[1].imshow(img_generated, aspect='auto')
        axarr[2].set_title("HR")
        axarr[2].grid(False)
        axarr[2].imshow(img_hr, aspect='auto')
        f.savefig(self.path_res + '/' + str(
            epoch + self.previous) + '.png', dpi=150, bbox_inches='tight')
        plt.close(f)
        imageio.imwrite(self.path_pure + '/' + str(epoch + self.previous) + '.png', img_generated)


In [9]:
from keras.callbacks import ModelCheckpoint

create_model = True
previous_steps = 0
epoch_to_launch = 5000
launch_in_night = False
partial_image_saving = 50
partial_model_saving = 4
step_for_epoch = 10
validation_steps = 2
path = "D:/GoogleDrive/Machine_Learning_Project-Classification+SRCNN/secondTaskData/" #change with ypur path (sharedFolder->secondTaskData
train_generator, validation_generator = get_image_generators()
model = CNN(create_model=create_model)

model_saver = ModelCheckpoint(path + 'temporaryModel.h5', monitor='val_loss', verbose=0,
                              save_best_only=False, save_weights_only=False, mode='auto', period=partial_model_saving)

#COMMENT THE ABOVE LINE IF YOU DO NOT WANT TO SAVE THE PARTIAL IMAGE RESULT AND REMOVE IT IN THE CALLBACKS IN FIT
callback_saver = PartialSaver(partial_image_saving, previous_steps, path)
print("to be starting......")
history = model.fit_generator(train_generator,
                              steps_per_epoch=step_for_epoch,
                              epochs=epoch_to_launch,
                              validation_data=validation_generator,
                              validation_steps=validation_steps, verbose=0,
                              callbacks=[callback_saver, model_saver])
print("terminated training")
print("begin saving.....")
save_model(model, path, name="GeneratorFitNew")
save_result(previous_steps + epoch_to_launch, path)
print("end saving. Completed")
if launch_in_night:
    os.system("shutdown /s")


Found 800 images belonging to 1 classes.
Found 800 images belonging to 1 classes.
Found 100 images belonging to 1 classes.


Found 100 images belonging to 1 classes.
CREATION MODEL


generator
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 256, 256, 64)      15616     
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 256, 256, 64)      36928     
_________________________________________________________________
batch_normalization_11 (Batc (None, 256, 256, 64)      256       
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 256, 256, 64)      36928     
_________________________________________________________________
batch_normalization_12 (Batc (None, 256, 256, 64)      256       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 256, 256, 64)      36928     


to be starting......
